In [52]:
import json
from tqdm.notebook import tqdm
from collections import Counter

In [53]:
filename = "correct.txt"

In [54]:
data = []

In [55]:
with open(filename, "r") as fin:
    for line in tqdm(fin):
        obj = json.loads(line)
        
        if "abstract" in obj and "year" in obj:
            if obj["year"] > 2020:
                data.append(obj["abstract"])

0it [00:00, ?it/s]

In [56]:
len(data)

27226

In [57]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from string import punctuation
import re
english_stopwords = stopwords.words("english")
lemmatizer = WordNetLemmatizer()


def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None


# Удаление знаков пунктуации из текста
def remove_punct(text):
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ', 41: ' ', 42: ' ', 43: ' ', 44: ' ', 45: ' ', 46: ' ', 47: ' ', 58: ' ', 59: ' ', 60: ' ', 61: ' ', 62: ' ', 63: ' ', 64: ' ', 91: ' ', 92: ' ', 93: ' ', 94: ' ', 95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

def preprocess(data):
    data = map(lambda x: x.lower(), data)
    data = map(lambda x: remove_punct(x), data)
    data = map(lambda x: re.sub(r'\d+', ' ', x), data)
    
    data = map(lambda x: x.split(' '), data)
    data = map(lambda x: [token for token in x if token not in english_stopwords\
                                                                      and token != " " \
                                                                      and token.strip() not in punctuation], data)
    
    data = map(lambda x: ' '.join(x), data)
    
    data = list(data)
    
    result = []
    for every in tqdm(data):
        pos_tagged = nltk.pos_tag(nltk.word_tokenize(every))
        wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))

        lemmatized_sentence = []
        for word, tag in wordnet_tagged:
            if tag is None:
                lemmatized_sentence.append(word)
            else:
                lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
        lemmatized_sentence = " ".join(lemmatized_sentence)

        result.append(lemmatized_sentence)
    
    return result

[nltk_data] Downloading package stopwords to /home/viliar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/viliar/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/viliar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/viliar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/viliar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [58]:
clean_data = preprocess(data)

  0%|          | 0/27226 [00:00<?, ?it/s]

In [59]:
len(clean_data)

27226

In [11]:
import pickle

In [12]:
with open("clean_data_abstract.pkl", "wb") as fout:
    pickle.dump(clean_data, fout)

### LDA

In [60]:
list_of_list_of_tokens = list(map(lambda x: x.split(' '), clean_data))

In [84]:
for i in range(len(list_of_list_of_tokens)):
    list_of_list_of_tokens[i] = list(filter(lambda x: len(x) > 3, list_of_list_of_tokens[i]))

In [85]:
from gensim import corpora, models

dictionary_LDA = corpora.Dictionary(list_of_list_of_tokens)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in list_of_list_of_tokens]

In [86]:
num_topics = 20
%time lda_model = models.LdaModel(corpus, num_topics=num_topics, \
                                  id2word=dictionary_LDA, \
                                  passes=4, alpha=[0.01]*num_topics, \
                                  eta=[0.01]*len(dictionary_LDA.keys()))

CPU times: user 1min 3s, sys: 78.4 ms, total: 1min 3s
Wall time: 1min 4s


In [89]:
for i, data in lda_model.show_topics(num_topics=20, formatted=False):
    for word, p in sorted(data, key=lambda x: x[1], reverse=True):
        print(f"{word}", end=" ")
    print("\n---------")

learn data model method task train label feature datasets base 
---------
social study user interaction group human effect participant behavior result 
---------
signal channel frequency system power performance communication base scheme user 
---------
detection image feature object method information segmentation base spatial attention 
---------
task video motion face track frame camera action http temporal 
---------
system device base service security attack vehicle data cloud application 
---------
system control time state robot dynamic design base controller paper 
---------
sensor patient measurement study health disease measure covid test time 
---------
function problem bound give paper show result abstract property class 
---------
graph problem time number cost algorithm show edge optimal game 
---------
product study policy change production chain market supply impact factor 
---------
data research system analysis paper provide approach process application base 
--------

In [90]:
lda_model.save('lda.model')

In [95]:
model =  models.LdaModel.load('lda.model')

# print all topics
model.show_topics(num_topics=20)

[(0,
  '0.038*"learn" + 0.029*"data" + 0.024*"model" + 0.019*"method" + 0.016*"task" + 0.013*"train" + 0.011*"label" + 0.011*"feature" + 0.011*"datasets" + 0.010*"base"'),
 (1,
  '0.018*"social" + 0.018*"study" + 0.018*"user" + 0.012*"interaction" + 0.012*"group" + 0.011*"human" + 0.008*"effect" + 0.008*"participant" + 0.008*"behavior" + 0.007*"result"'),
 (2,
  '0.023*"signal" + 0.020*"channel" + 0.015*"frequency" + 0.014*"system" + 0.013*"power" + 0.013*"performance" + 0.011*"communication" + 0.011*"base" + 0.010*"scheme" + 0.010*"user"'),
 (3,
  '0.025*"detection" + 0.023*"image" + 0.021*"feature" + 0.021*"object" + 0.017*"method" + 0.011*"information" + 0.011*"segmentation" + 0.010*"base" + 0.009*"spatial" + 0.008*"attention"'),
 (4,
  '0.034*"task" + 0.032*"video" + 0.027*"motion" + 0.020*"face" + 0.018*"track" + 0.018*"frame" + 0.017*"camera" + 0.014*"action" + 0.013*"http" + 0.013*"temporal"'),
 (5,
  '0.028*"system" + 0.014*"device" + 0.014*"base" + 0.014*"service" + 0.013*"sec